# 12. 고급 NumPy
> ## 고급 ufunc 사용법

In [1]:
import numpy as np
import pandas as pd
from numpy.random import randn

___
## 1. ufunc 인스턴스 메서드

In [2]:
arr = np.arange(10)

np.add.reduce(arr)

45

In [4]:
arr.sum()

45

- reduce는 축에따라 값을 집계해 줌

In [6]:
arr = randn(5,5)
arr[::2].sort(1)
arr[:, :-1] < arr[:, 1:]

array([[ True,  True,  True,  True],
       [False,  True, False, False],
       [ True,  True,  True,  True],
       [ True, False,  True, False],
       [ True,  True,  True,  True]])

In [8]:
np.logical_and.reduce(arr[:, :-1] < arr[:, 1:], axis = 1)

array([ True, False,  True, False,  True])

- logical_and.reduce는 all 메서드와 동일

In [9]:
arr = np.arange(15).reshape(3,5)
np.add.accumulate(arr, axis = 1)

array([[ 0,  1,  3,  6, 10],
       [ 5, 11, 18, 26, 35],
       [10, 21, 33, 46, 60]], dtype=int32)

In [11]:
arr.cumsum(axis = 1)

array([[ 0,  1,  3,  6, 10],
       [ 5, 11, 18, 26, 35],
       [10, 21, 33, 46, 60]], dtype=int32)

- accumulate는 reduce 메서드와 관련이 있음
- cumsum() 메서드와 유사

In [12]:
arr = np.arange(3).repeat([1,2,2])
arr

array([0, 1, 1, 2, 2])

In [13]:
np.multiply.outer(arr, np.arange(5))

array([[0, 0, 0, 0, 0],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 2, 4, 6, 8],
       [0, 2, 4, 6, 8]])

- multiply.outer는 외적을 계산

In [14]:
result = np.subtract.outer(randn(3,4), randn(5))
result.shape

(3, 4, 5)

In [15]:
result

array([[[-0.28335911, -0.26553832,  0.66980207, -0.91548274,
         -1.45710294],
        [ 1.70018809,  1.71800888,  2.65334927,  1.06806446,
          0.52644426],
        [-0.23799363, -0.22017284,  0.71516755, -0.87011726,
         -1.41173746],
        [ 1.36261506,  1.38043584,  2.31577624,  0.73049142,
          0.18887122]],

       [[-1.10948777, -1.09166698, -0.15632659, -1.7416114 ,
         -2.28323161],
        [ 0.08971759,  0.10753837,  1.04287877, -0.54240605,
         -1.08402625],
        [-0.38468393, -0.36686315,  0.56847725, -1.01680757,
         -1.55842777],
        [ 1.02640627,  1.04422706,  1.97956745,  0.39428264,
         -0.14733756]],

       [[ 0.50488455,  0.52270533,  1.45804573, -0.12723909,
         -0.66885929],
        [ 0.80852898,  0.82634976,  1.76169016,  0.17640534,
         -0.36521486],
        [ 0.65846673,  0.67628751,  1.61162791,  0.02634309,
         -0.51527711],
        [-1.01265847, -0.99483769, -0.05949729, -1.64478211,
         -2

- outer 메서드 결과의 차원은 입력된 차원의 합이 됨

In [16]:
arr = np.arange(10)
np.add.reduceat(arr,[0, 5, 8])

array([10, 18, 17], dtype=int32)

In [20]:
[arr[0:5].sum(), arr[5:8].sum(), arr[8:].sum()]

[10, 18, 17]

- reduceat은 정해진 구역별로 색인하여 합계 반환

In [22]:
arr = np.multiply.outer(np.arange(4), np.arange(5))
arr

array([[ 0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4],
       [ 0,  2,  4,  6,  8],
       [ 0,  3,  6,  9, 12]])

In [23]:
np.add.reduceat(arr, [0, 2, 4], axis = 1)

array([[ 0,  0,  0],
       [ 1,  5,  4],
       [ 2, 10,  8],
       [ 3, 15, 12]], dtype=int32)

- column 단위 색인도 가능
___
## 2. 사용자 ufunc

In [24]:
def add_elements(x, y):
    return x + y

add_them = np.frompyfunc(add_elements, 2, 1)

add_them(np.arange(8), np.arange(8))

array([0, 2, 4, 6, 8, 10, 12, 14], dtype=object)

- frompyfunc를 이용해서 생성한 함수는 항상 파이썬 객체가 담긴 배열을 반환

In [25]:
add_them = np.vectorize(add_elements, otypes = [np.float64])
add_them(np.arange(8), np.arange(8))

array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14.])

- np.vectorize 함수는 자료형을 추론하나 기능이 미흡

In [26]:
arr = randn(10000)

%timeit add_them(arr, arr)

1.6 ms ± 31.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [27]:
%timeit np.add(arr, arr)

3.34 µs ± 12 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


- ufunc 스타일의 함수는 각 원소를 계산하기 위해 파이썬 함수를 호출 (따라서 느림)
- NumPy의 C 기반 ufunc 반복문보다 많이 느림